# CNN with keras

In [1]:
# !pip install tensorflow -q
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler

import os
import shutil
import glob
import random
import shutil
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)
%matplotlib inline


2022-10-27 20:13:41.349605: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-27 20:13:41.349658: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-27 20:13:41.453488: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-27 20:13:43.201249: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-10-27 20:13:43.201428: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: ca

In [2]:
# physical_devices=tf.config.experimental.list_physical_devices('GPU')
# print(f"number of GPU available {len(physical_devices)}, named {physical_devices}")
# tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [3]:
# download Kaggle: Cat VS Dog Dataset

# !wget https://www.kaggle.com/datasets/karakaggle/kaggle-cat-vs-dog-dataset?resource=download

In [14]:
from genericpath import isdir
import os
print(os.path.abspath('.'))
os.chdir("PetImages")
print(os.path.abspath('.'))


folders=['train','valid','test']
classification=['dog','cat']
print(os.path.basename(__name__))
if os.path.isdir('train/dog') is False:
    for a in folders:
        for b in classification:
            temppath=f"{a}/{b}"
            if os.isdir(temppath) is False:
            os.makedirs(temppath)
            

            for img in random.sample(glob.glob("Cat/*.jpg"),500):
                shutil.copy(img,temppath)

/home/prateek/Desktop/ML from Scratch/CNN-with-Keras/Part2/PetImages


FileNotFoundError: [Errno 2] No such file or directory: 'PetImages'

In [ ]:
import os
